In [1]:
import random
import time


In [2]:
class Node:
    def __init__(self, key, value):
        self.data = (key, value)
        self.next = None
        self.prev = None

In [4]:
class DoublyLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def push(self, key, value):
        new_node = Node(key, value)
        new_node.next = self.head
        if self.head:
            self.head.prev = new_node
        else:
            self.tail = new_node
        self.head = new_node
        return new_node

    def remove(self, node):
        if node.prev:
            node.prev.next = node.next
        else:
            self.head = node.next
        if node.next:
            node.next.prev = node.prev
        else:
            self.tail = node.prev
        node.prev = None
        node.next = None

    def move_to_front(self, node):
        if node != self.head:
            self.remove(node)
            node.next = self.head
            self.head.prev = node
            self.head = node

    def remove_last(self):
        if self.tail:
            last = self.tail
            self.remove(last)
            return last
        return None

class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache = {}
        self.list = DoublyLinkedList()

    def get(self, key):
        if key in self.cache:
            node = self.cache[key]
            self.list.move_to_front(node)
            return node.data[1]
        return -1

    def put(self, key, value):
        if key in self.cache:
            node = self.cache[key]
            node.data = (key, value)
            self.list.move_to_front(node)
        else:
            if len(self.cache) >= self.capacity:
                last = self.list.remove_last()
                if last:
                    del self.cache[last.data[0]]
            new_node = self.list.push(key, value)
            self.cache[key] = new_node


In [5]:
# Функції без кешування
def range_sum_no_cache(array, left, right):
    return sum(array[left : right + 1])

In [6]:
def update_no_cache(array, index, value):
    array[index] = value


In [7]:
def range_sum_with_cache(array, left, right, cache):
    key = (left, right)

    # Спроба отримати значення з кешу
    val = cache.get(key)

    if val != -1:
        return val

    # Промах кешу - обчислення суми
    total = sum(array[left : right + 1])
    cache.put(key, total) # оновлення кешу
    return total

In [8]:
def update_with_cache(array, index, value, cache):
    # 1. Оновлюємо масив
    array[index] = value

    # 2. Інвалідація: знаходимо ключі, які зачіпає цей індекс
    keys_to_remove = []

    for key in cache.cache:
        (L, R) = key
        if L <= index <= R:
            keys_to_remove.append(key)

    # Видаляємо знайдені вузли і з словника, і зі зв'язного списку
    for key in keys_to_remove:
        node_to_remove = cache.cache[key]
        # Видаляємо зі списку
        cache.list.remove(node_to_remove)
        # Видаляємо зі словника
        del cache.cache[key]

In [9]:
def make_queries(n, q, hot_pool=30, p_hot=0.95, p_update=0.03):
    hot = [(random.randint(0, n//2), random.randint(n//2, n-1))
           for _ in range(hot_pool)]
    queries = []
    for _ in range(q):
        if random.random() < p_update:
            idx = random.randint(0, n-1)
            val = random.randint(1, 100)
            queries.append(("Update", idx, val))
        else:
            if random.random() < p_hot:
                left, right = random.choice(hot)
            else:
                left = random.randint(0, n-1)
                right = random.randint(left, n-1)
            queries.append(("Range", left, right))
    return queries

In [10]:
if __name__ == "__main__":
    N = 100_000
    Q = 50_000
    CACHE_CAPACITY = 1000

    print(f"Генерація даних: масив {N}, запитів {Q}...")
    original_array = [random.randint(1, 100) for _ in range(N)]
    queries = make_queries(N, Q)

    # Без кешу
    print("Тест БЕЗ кешу...")
    arr_no_cache = original_array[:]

    start_time = time.time()
    for query in queries:
        if query[0] == "Range":
            range_sum_no_cache(arr_no_cache, query[1], query[2])
        elif query[0] == "Update":
            update_no_cache(arr_no_cache, query[1], query[2])
    end_time = time.time()
    time_no_cache = end_time - start_time

    # З LRU-кешем
    print("Тест з LRU-кешем...")
    arr_with_cache = original_array[:]
    lru = LRUCache(CACHE_CAPACITY)
    start_time = time.time()
    for query in queries:
        if query[0] == "Range":
            range_sum_with_cache(arr_with_cache, query[1], query[2], lru)
        elif query[0] == "Update":
            update_with_cache(arr_with_cache, query[1], query[2], lru)
    end_time = time.time()
    time_with_cache = end_time - start_time

    print("-" * 40)
    print(f"Без кешу : {time_no_cache:6.2f} c")
    print(f"LRU-кеш  : {time_with_cache:6.2f} c  (прискорення ×{time_no_cache / time_with_cache:.1f})")
    print("-" * 40)

Генерація даних: масив 100000, запитів 50000...
Тест БЕЗ кешу...
Тест з LRU-кешем...
----------------------------------------
Без кешу :  25.91 c
LRU-кеш  :   9.73 c  (прискорення ×2.7)
----------------------------------------
